In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('odi_bbb_2005_2024.csv')

C:\Users\Vijay\AppData\Local\Temp\ipykernel_2168\2762841208.py:1: DtypeWarning: Columns (40,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('odi_bbb_2005_2024.csv')


Since I've imported the dataset and performed several operations on it already, I will just pick the required columns of the entire available columns

In [3]:
df.columns

Index(['p_match', 'inns', 'bat', 'p_bat', 'team_bat', 'bowl', 'p_bowl',
       'team_bowl', 'ball', 'ball_id', 'outcome', 'score', 'out', 'dismissal',
       'p_out', 'over', 'noball', 'wide', 'byes', 'legbyes', 'cur_bat_runs',
       'cur_bat_bf', 'cur_bowl_ovr', 'cur_bowl_wkts', 'cur_bowl_runs',
       'inns_runs', 'inns_wkts', 'inns_balls', 'inns_runs_rem',
       'inns_balls_rem', 'inns_rr', 'inns_rrr', 'target', 'max_balls', 'date',
       'year', 'ground', 'country', 'winner', 'toss', 'competition',
       'bat_hand', 'bowl_style', 'bowl_kind', 'batruns', 'ballfaced',
       'bowlruns', 'bat_out', 'wagonX', 'wagonY', 'wagonZone', 'line',
       'length', 'shot', 'control', 'predscore', 'wprob'],
      dtype='object')

In [4]:
req_cols=['date','bat','inns_runs','inns_wkts']
df.isnull().sum()

p_match                 0
inns                    0
bat                     0
p_bat                   0
team_bat                0
bowl                    0
p_bowl                  0
team_bowl               0
ball                    0
ball_id                 0
outcome                 0
score                   0
out                     0
dismissal         1286194
p_out                   0
over                    0
noball                  0
wide                    0
byes                    0
legbyes                 0
cur_bat_runs            0
cur_bat_bf              0
cur_bowl_ovr            0
cur_bowl_wkts           0
cur_bowl_runs           0
inns_runs               0
inns_wkts               0
inns_balls              0
inns_runs_rem      716650
inns_balls_rem          0
inns_rr                 0
inns_rrr           716650
target             716650
max_balls               0
date                    0
year                    0
ground                  0
country                 0
winner      

In [5]:
df=df[req_cols]

Now, the aim is to find the min/max count of wickets and runs when a player takes strike. It gives us our required data to visualize

In [6]:
df['entry']=df.groupby(by=['date','bat'])['inns_runs'].transform('min')
df['arrive_fow']=df.groupby(by=['date','bat'])['inns_wkts'].transform('min')
df['exit_fow']=df.groupby(by=['date','bat'])['inns_wkts'].transform('max')
df['exit']=df.groupby(by=['date','bat'])['inns_runs'].transform('max')

Our Added is basically the number of runs added to the team's total when the player is at the crease

In [7]:
df['added']=df['exit']-df['entry']

Finding the size will give me the total number of balls faced by a batter in each match, but I want the innings count, so I will map each batter and date combination to 1 which denotes the number of innings.

In [8]:
ans = df.groupby(['bat', 'date']).size().reset_index(name='innings')
ans['innings']=1

With my dataset being ready to do the mathematical calculations, all I have to do is find the mean of arrival and exit team score and wickets.

In [9]:
res=df.groupby('bat')['bat','entry','arrive_fow','exit','exit_fow','added'].mean().reset_index()
res=res.sort_values(by=['added'],ascending=False)

C:\Users\Vijay\AppData\Local\Temp\ipykernel_2168\1525163562.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  res=df.groupby('bat')['bat','entry','arrive_fow','exit','exit_fow','added'].mean().reset_index()
C:\Users\Vijay\AppData\Local\Temp\ipykernel_2168\1525163562.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res=df.groupby('bat')['bat','entry','arrive_fow','exit','exit_fow','added'].mean().reset_index()


In [10]:
res

,bat,entry,arrive_fow,exit,exit_fow,added
724,John Campbell,1.087156,0.000000,250.784404,1.000000,249.697248
376,Dawid Malan,15.362225,0.599612,196.156533,3.711514,180.794308
677,Janneman Malan,0.672897,0.000850,180.141886,2.982158,179.468989
605,Ibrahim Zadran,3.537037,0.038126,173.098039,2.846405,169.561002
144,Asa Tribe,44.771341,2.951220,211.600610,5.198171,166.829268
...,...,...,...,...,...,...
359,Darren Ramsammy,175.000000,9.000000,175.000000,9.000000,0.000000
343,Daniel Birrell,218.000000,10.000000,218.000000,10.000000,0.000000
1671,William O’Rourke,98.000000,9.000000,98.000000,10.000000,0.000000
526,Graeme McCarter,241.000000,9.000000,241.000000,9.000000,0.000000


In [11]:
innings=ans.groupby(by=['bat']).sum().reset_index()

C:\Users\Vijay\AppData\Local\Temp\ipykernel_2168\282506693.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  innings=ans.groupby(by=['bat']).sum().reset_index()


In [12]:
innings

,bat,innings
0,AB de Villiers,218
1,AM Ghazanfar,1
2,Aamer Yamin,3
3,Aamir Kaleem,2
4,Aarif Sheikh,43
...,...,...
1707,Zubayr Hamza,1
1708,Zubin Surkari,20
1709,Zulfiqar Babar,5
1710,Zulqarnain Haider,4


With innings count and entry arrival stats being in different datasets, I will merge them into a single dataset by having the 'bat' as common index.

In [13]:
final=pd.merge(innings,res,on='bat')

In [14]:
final['entry']=round(final['entry'],0)
final['exit']=round(final['exit'],0)
final['added']=round(final['added'],0)

In [15]:
final.to_csv('runs_added.csv')